In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/project/section-2/mlruns/1', creation_time=1719853039311, experiment_id='1', last_update_time=1719853039311, lifecycle_stage='active', name='project-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1719852963603, experiment_id='0', last_update_time=1719852963603, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.accuracy < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.accuracy ASC"]
)

In [10]:
for run in runs:
    print(f"run id: {run.info.run_id}, accuracy: {run.data.metrics['accuracy']:.4f}")

run id: 45490009a7864814b3b4b0fdad5a8d52, accuracy: 0.9393
run id: f91b137207d74e22961f8df62938ea96, accuracy: 0.9393
run id: af1dc8c7e1d44490abcd244e7e95e2ca, accuracy: 0.9393
run id: efef04d3fc3a4bee94cd9c03648f82e2, accuracy: 0.9393
run id: 60f0b5e5336f474c8e034e32e9275f1e, accuracy: 0.9393


Interacting with Model Registry

In [11]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
run_id = "45490009a7864814b3b4b0fdad5a8d52"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="project-health-stroke")

Successfully registered model 'project-health-stroke'.
Created version '1' of model 'project-health-stroke'.


<ModelVersion: aliases=[], creation_timestamp=1719856861826, current_stage='None', description=None, last_updated_timestamp=1719856861826, name='project-health-stroke', run_id='45490009a7864814b3b4b0fdad5a8d52', run_link=None, source='/workspaces/project/section-2/mlruns/1/45490009a7864814b3b4b0fdad5a8d52/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [13]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1719856861759, description=None, last_updated_timestamp=1719856861826, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1719856861826, current_stage='None', description=None, last_updated_timestamp=1719856861826, name='project-health-stroke', run_id='45490009a7864814b3b4b0fdad5a8d52', run_link=None, source='/workspaces/project/section-2/mlruns/1/45490009a7864814b3b4b0fdad5a8d52/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='project-health-stroke', tags={}>]

Verify the models registered wrt its stages

In [14]:
model_name = "project-health-stroke"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_32597/3731493744.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


Manually transition the stage of particular version of model

In [15]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_32597/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1719856861826, current_stage='Staging', description=None, last_updated_timestamp=1719857018484, name='project-health-stroke', run_id='45490009a7864814b3b4b0fdad5a8d52', run_link=None, source='/workspaces/project/section-2/mlruns/1/45490009a7864814b3b4b0fdad5a8d52/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

Update the registered models with its transition details wrt date and time

It is like a description

In [16]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1719856861826, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-07-01', last_updated_timestamp=1719857047766, name='project-health-stroke', run_id='45490009a7864814b3b4b0fdad5a8d52', run_link=None, source='/workspaces/project/section-2/mlruns/1/45490009a7864814b3b4b0fdad5a8d52/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

Check the staged version code rmse is predicted good with the test data and move to production

we can move to prod and also can check the predictions

In [ ]:
'''
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)
'''